In [1]:
import ase
from ase import Atoms
from ase.visualize import view
import numpy as np

In [12]:
def get_chemical_formula(A,B,X,n_array):
    
    x = str(np.sum(n_array+1))
    y = str(np.sum(n_array))
    z = str(np.sum(3*(n_array)+1))

    return A+x+B+y+X+z

def perovskite_layer(A,B,X):
    return Atoms([A,B,X,X,X], 
           positions=[(0.5,0.5,0.5),(0,0,0),(0.5,0.5,0),(0.5,0,0.5),(0,0.5,0.5)])

def rocksalt_layer(A,X):
    return Atoms([A,X], 
           positions=[(0,0,0),(0.5,0.5,0)])

def create_rp_structure(A,B,X,n_array,a,b, perovskite_spacing,rocksalt_spacing):
    total_length = np.sum(n_array)*perovskite_spacing + len(n_array)*rocksalt_spacing
    rp_structure = Atoms(cell=[a,b,total_length,101,101,90])

    current_c_position = 0
    for number_of_layers in n_array:
        for layer_count in range(number_of_layers):
            rp_structure += perovskite_layer(A,B,X).translate([0,0,current_c_position])
            current_c_position += perovskite_spacing
        rp_structure += rocksalt_layer(A,X).translate([0,current_c_position])
        current_c_position += rocksalt_spacing
        
    return rp_structure

def rattle_structure(Atoms):

    return Atoms.rattle(stdev=0.02, seed=1)

def save_structure(Atoms,n_array,A,B,X):

    ase.io.write('RP_{}_{}.cif'.format(''.join(map(str,n_array)),A+B+X), Atoms)

In [3]:
t = get_chemical_formula("Ba","Zr","S",np.array([2,1]))

In [38]:
perov = perovskite_layer("Ba","Zr","S")
view(perov, viewer='ngl')

In [4]:
t

'Ba5Zr3S11'

In [39]:
rs = rocksalt_layer("Ba","S")
view(rs, viewer='ngl')

In [14]:
atoms = create_rp_structure('Ba','Zr','S',np.array([2,1]),3,3,3,2)

TypeError: object of type 'NoneType' has no len()

In [19]:
t=perovskite_layer('Ba','Zr','S')


In [22]:
t.translate([2,0,0])